# Google Maps Photos Downloader
- Using [Google Maps Places API (New)](https://developers.google.com/maps/documentation/places/web-service/op-overview)

## Getting started
- Create a Google Cloud project, enable the Places API (New)
- Create an API key and paste it below

In [ ]:
# API_KEY="REDACTED"

In [11]:
!pip3 install googlemaps

import googlemaps
import requests
import json


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [19]:
# Read all place names from yale_poi.txt
with open("yale_poi.txt", "r") as f:
    poi_names = f.readlines()

# Strip newline characters
poi_names = [x.strip() for x in poi_names]
print(poi_names)

['1 Church Street', '10 Amistad Street', '10 Sachem Street', '100 Church Street South', '100 College Street', '100 York Street', '104 York Square Place', '1201 Chapel Street', '1203 Chapel Street', '121 Whitney Avenue', '127 College Street', '129 York Street', '132 Davenport Avenue', '135 College Street', '143 Elm Street', '149 Elm Street', '149 York Street', '15 Hillhouse Avenue', '150 York Street', '153 College Street', '157 Church Street', '160 St. Ronan Street', '17 Hillhouse Avenue', '175 St. Ronan Street', '195 Church Street', '2 Church Street South', '2 Science Park', '202 Prospect Street', '202 York Street', '204 Prospect Street', '204 York Street', '205 Park Street', '205 Whitney Avenue', '210 Prospect Street', '211 Park Street', '212 York Street', '215 Park Street', '217 Park Street', '220 West Campus Drive', '220 York Street', '221 Whitney Avenue', '230 Prospect Street', '238 Prospect Street', '24 Hillhouse Avenue', '240 West Campus Drive', '242 Prospect Street', '246 Church

## Getting the Place ID
- Use [this tool](https://developers.google.com/maps/documentation/places/web-service/place-id) to find the Place ID of Yale Unviersity
- Place ID: `ChIJRUlizbbZ54kRJwTTtMSiNK4`

In [ ]:
gmaps = googlemaps.Client(key=API_KEY)

places = []

for poi in poi_names:
    query = poi + ", New Haven, CT"
    places_result = gmaps.places(query=query)["results"]
    
    if len(places_result) == 0:
        print(f"No results found for {poi}")
        continue

    top_result = places_result[0]
    print(top_result["name"])

    places.append(top_result)

1 Church
10 Amistad St
10 Sachem St
100 Church St S
100 College St
100 York St
104 York Square Pl
1201 Chapel St
1203 Chapel St
121 Whitney Ave
127 College St
Crown Towers
132 Davenport Ave
135 College St
143 Elm St
James Pierpont House
149 York St
15 Hillhouse Ave
Chapel-York Garage
153 College St
157 Church St
160 St Ronan St
17 Hillhouse Ave
175 St Ronan St
195 Church St
2 Church Street South
2 Science Park
202 Prospect St
202 York St
204 Prospect St
204 York St
205 Park St
205 Whitney Ave
210 Prospect St
211 Park St
212 York St
215 Park St
217 Park St
220 West Campus Drive
220 York St
221 Whitney Ave
230 Prospect St
Yale Interdisciplinary Center for Bioethics
24 Hillhouse Ave
240 West Campus Drive
242 Prospect St
246 Church St
25 Science Park
250 Church St
250 West Campus Drive
254 Elm St
254 Prospect St
258 Church St
26 High St
Economic Growth Center
270 Congress Ave
270 Crown St
Parking lot
276 Prospect St
286 York St #276
279 Mansfield St
Charles Henry Farnam House
282 Prospect 

In [24]:
photos = []

def get_photos(place_id, name):
    details = gmaps.place(place_id)["result"]
    place_photos = details.get("photos", [])
    photos.extend(place_photos)
    print(f"Found {len(place_photos)} photos for {name}")

for place in places:
    get_photos(place["place_id"], place["name"])

# Also get for Yale University
get_photos("ChIJRUlizbbZ54kRJwTTtMSiNK4", "Yale University")

Found 0 photos for 1 Church
Found 0 photos for 10 Amistad St
Found 0 photos for 10 Sachem St
Found 0 photos for 100 Church St S
Found 1 photos for 100 College St
Found 0 photos for 100 York St
Found 0 photos for 104 York Square Pl
Found 0 photos for 1201 Chapel St
Found 0 photos for 1203 Chapel St
Found 0 photos for 121 Whitney Ave
Found 0 photos for 127 College St
Found 0 photos for Crown Towers
Found 0 photos for 132 Davenport Ave
Found 0 photos for 135 College St
Found 0 photos for 143 Elm St
Found 0 photos for James Pierpont House
Found 0 photos for 149 York St
Found 0 photos for 15 Hillhouse Ave
Found 0 photos for Chapel-York Garage
Found 0 photos for 153 College St
Found 0 photos for 157 Church St
Found 2 photos for 160 St Ronan St
Found 0 photos for 17 Hillhouse Ave
Found 0 photos for 175 St Ronan St
Found 0 photos for 195 Church St
Found 0 photos for 2 Church Street South
Found 3 photos for 2 Science Park
Found 0 photos for 202 Prospect St
Found 0 photos for 202 York St
Found 0

In [25]:
# Dedupe photos
photo_urls = set()
for photo in photos:
    photo_urls.add(photo["photo_reference"])

photo_urls = list(photo_urls)
print(len(photo_urls))

728


In [ ]:
photo_references = [photo['photo_reference'] for photo in photos]

for i, photo_reference in enumerate(photo_references):
    photo_url = f"https://maps.googleapis.com/maps/api/place/photo?maxwidth=400&photoreference={photo_reference}&key={API_KEY}"

    response = requests.get(photo_url)
    response.raise_for_status()
    print(f"Downloaded {photo_url}")
    with open(f"download/{i}.jpg", "wb") as f:
        f.write(response.content)

print(f"Downloaded {len(photo_references)} photos.")